In [ ]:
from pathlib import Path
import numpy as np
import torch
import math
from tqdm import tqdm
from torchvision.datasets import CocoDetection
import matplotlib.pyplot as plt
import torchvision

In [ ]:
from abbyy_course_cvdl_t3.coco_text import COCO_Text
from abbyy_course_cvdl_t3 import coco_evaluation
from abbyy_course_cvdl_t3.utils import evaluate_ap_from_cocotext_json
from abbyy_course_cvdl_t3.utils import dump_detections_to_cocotext_json

In [ ]:
cd ../task2

import abbyy_course_cvdl_t2
from abbyy_course_cvdl_t2.convert import PointsToObjects, ObjectsToPoints
from abbyy_course_cvdl_t2.impl.train import train
from abbyy_course_cvdl_t2.impl.data import CocoTextDetection, CocoDetectionPrepareTransform
from abbyy_course_cvdl_t2.network import CenterNet
from abbyy_course_cvdl_t2.loss import CenterNetLoss

cd ../task3

In [ ]:
device = torch.device('cuda:0')

In [ ]:
net = torch.load('centernet.pth').to(device)
anno = torch.load('anno.pt')
print(anno.shape)
inputs = torch.load('inputs.pt')
print(inputs.shape)
outputs = torch.load('outputs.pt')
print(outputs.shape)

In [ ]:
for p in list(net.parameters()):
    if p.isnan().any():
        print(p)

In [ ]:
with torch.no_grad():
    out = net(torch.from_numpy(inputs[0])[None, ...].to(device)).cpu()
print(out[0].isnan().any())

In [ ]:
with torch.no_grad():
    backbone_out = net.backbone(torch.from_numpy(inputs[0])[None, ...].to(device)).cpu()
print(backbone_out.isnan().any())

In [ ]:
with torch.no_grad():
    encoder_out = net.backbone.downscale(torch.from_numpy(inputs[0])[None, ...].to(device)).cpu()
print(encoder_out.isnan().any())

In [ ]:
y = torch.from_numpy(inputs).to(device)
print(y)

with torch.no_grad():
    for layer in list(net.backbone.downscale.md.children()):
        x = layer(y)

        if x.isnan().any() or x.isinf().any():
            print(layer, x.shape, y.shape)
            break

        y = x.clone()